In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from pathlib import Path
import pandas as pd
import numpy as np
from spotifuncs import *

In [2]:
path = Path("C:/Users/ms101/OneDrive/DataScience_ML/projects/spotify_app")

In [3]:
with open(path / "client_s.txt") as f:
    content = f.readlines()
content = [x.strip() for x in content]

client_id = content[0]
client_secret = content[1]

In [4]:
with open(path / "usernames.txt") as f:
    usernames = f.readlines()
usernames = [x.strip() for x in usernames]

username1 = usernames[0]

In [5]:
scope = "user-library-read user-read-recently-played user-top-read playlist-modify-public playlist-modify-private playlist-read-private playlist-read-collaborative"

redirect_uri = "https://developer.spotify.com/dashboard/applications/4a4e029d299a4241873db8300038bf0a"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, 
                                                      client_secret=client_secret)


In [6]:
sp_m = authenticate(redirect_uri, client_credentials_manager, username1, scope, client_id, client_secret)

In [7]:
redlight = sp_m.playlist("spotify:playlist:3zcSUFp0puoWXWXuFCF2e6")

In [16]:
# def create_df_playlist(api_results,sp = None, append_audio = True):
#     df = create_df_saved_songs(api_results["tracks"])
#     if append_audio == True:
#         assert sp != None, "sp needs to be specified for appending audio features"
#         df = append_audio_features(df,sp)
#     return df

In [8]:
redlight_df = create_df_playlist(redlight)#see if assert error works

AssertionError: sp needs to be specified for appending audio features

In [9]:
redlight_df = create_df_playlist(redlight, sp = sp_m)
redlight_df

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Desert Woman,RINGOS DESERT,4O1nIKXTS87DKHIQejrn3n,ZHU,374610,44,0.710,0.651,1,-8.537,1,0.0707,0.022300,0.864,0.0657,0.1500,122.987
1,York - Original Mix,York,3eJBKLhpOauQJlgoOSPErD,Christian Löffler,476675,49,0.696,0.634,11,-12.063,0,0.0403,0.157000,0.887,0.1290,0.0359,118.035
2,Personal Space,Personal Space / Mulholland 99,3TYNQdnAM85I6P9UegzYKo,Yotto,301295,46,0.645,0.848,7,-6.913,1,0.0351,0.002540,0.956,0.0868,0.0383,122.011
3,October,October,6FIQ8o2hqlDmHQFoBKmKgW,Icarus,354102,59,0.365,0.711,8,-9.591,1,0.0355,0.030600,0.913,0.0824,0.1170,116.003
4,Azure,Berlin Calling (The Soundtrack by Paul Kalkbre...,2VnlJCQMphFJUyYR5p7da2,Paul Kalkbrenner,364059,57,0.730,0.504,4,-13.915,1,0.0477,0.124000,0.882,0.1070,0.0936,122.013
5,Eating Hooks - Siriusmo Remix - Solomun Edit,Eating Hooks (Siriusmo Remix - Solomun Edit),6fgmNIMwDdLQrNzu4U8pcU,Moderat,422200,55,0.661,0.728,8,-8.713,1,0.0379,0.008180,0.652,0.0599,0.3470,123.994
6,Parallaxis - Traumprinz's Over 2 The End Version,Decay Versions Pt. 2,6o6LRX4HHHGKgH7NuMxw6u,Efdemin,406000,42,0.688,0.496,7,-15.994,1,0.0519,0.076400,0.894,0.1170,0.2030,122.971
7,Mango,Mango,1lK8OlJuuoVsGHsWaKFAfk,Sascha Funke,420919,46,0.670,0.548,5,-11.342,0,0.0571,0.000724,0.918,0.0857,0.4380,124.996
8,Risky Business,Risky Business,75N5R8CeZDyJurQJsoRTx1,ZHU,320853,58,0.787,0.653,11,-6.825,0,0.0649,0.087500,0.735,0.0933,0.3960,122.995
9,Little by Little,Little by Little,2XgcOGlE73KWYa43Deq8OJ,Lane 8,333223,54,0.649,0.771,9,-9.714,1,0.0412,0.008590,0.853,0.1030,0.6080,121.011


In [12]:
cols = redlight_df.columns[6:].tolist()
cols

['danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo']

In [13]:
redlight_df[cols].mean()

danceability          0.733804
energy                0.566255
key                   6.215686
loudness            -11.472686
mode                  0.529412
speechiness           0.059265
acousticness          0.119777
instrumentalness      0.848980
liveness              0.113824
valence               0.250771
tempo               118.373902
dtype: float64

In [10]:
mean_song = pd.DataFrame(columns=redlight_df.columns)
mean_song.loc["mean"] = redlight_df.mean()

In [11]:
mean_song

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
mean,NaN,NaN,NaN,NaN,407060.784314,37.647059,0.733804,0.566255,6.215686,-11.472686,0.529412,0.059265,0.119777,0.84898,0.113824,0.250771,118.373902


In [12]:
seed_tracks = redlight_df["track_id"].tolist()

In [13]:
len(seed_tracks)

51

In [14]:
recomm_dfs = []
for i in range(5,len(seed_tracks)+1,5):
    recomms = sp_m.recommendations(seed_tracks = seed_tracks[i-5:i],limit = 25)
    recomms_df = append_audio_features(create_df_recommendations(recomms),sp_m)
    recomm_dfs.append(recomms_df)
recomms_df = pd.concat(recomm_dfs)

In [15]:
recomms_df

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Hausch,"Get Physical Music Presents: Body Language, Vo...",1bVNvJdZllDJzpQ68dq4Lg,Andhim,445749,51,0.805,0.827,2,-8.661,1,0.0360,0.040700,0.860000,0.2130,0.9430,120.005
1,This Is Just To Say,This Is Just To Say,2wiBgJOpUqE7qpJkPddNAV,Running Touch,293511,54,0.733,0.564,8,-9.435,1,0.0724,0.396000,0.000013,0.0734,0.4270,80.023
2,Hello Clouds - Original Mix,Hello Clouds,4E8XEz8KIuQ1iJJAMgCcXV,Justin Martin,318892,34,0.789,0.612,5,-10.123,0,0.0526,0.229000,0.507000,0.0746,0.4850,121.002
3,Tuur mang Welten,Tuur mang Welten,3vZ41qZR29470JVWKsdzsV,Niklas Paschburg,277597,52,0.290,0.347,1,-16.585,0,0.0439,0.966000,0.850000,0.3710,0.2530,179.921
4,Glitter (feat. Ambré Perkins),Cura,3pEWIQo3WBQNyMQRjEwAJG,Keys N Krates,214000,40,0.628,0.579,7,-7.945,1,0.1440,0.034300,0.004760,0.2340,0.6780,101.089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,They All Called 'Em Garys,They All Called 'Em Garys,7j3eLuQJOgQvDTbVAWzGWx,Alan Fitzpatrick,411219,43,0.682,0.902,7,-8.096,1,0.0556,0.001010,0.798000,0.1130,0.2970,127.007
21,FEEL NOTHING,VOL. 4 :: SLAVES OF FEAR,0V7joapUtnZmfLhLj1QVI4,HEALTH,179373,51,0.540,0.816,1,-7.334,1,0.0310,0.000049,0.010400,0.0806,0.2420,98.028
22,The Beat,The Beat,1wkDPTi5PdTMx9kZEy0oQ3,Sam Paganini,469603,33,0.700,0.817,7,-10.042,1,0.0471,0.008770,0.921000,0.0924,0.0386,128.019
23,Destroy Me,ETERNITY,4GFF8KTXPeL3Omy31aD8A4,Mr.Kitty,354103,52,0.587,0.680,10,-8.581,0,0.0590,0.001510,0.000012,0.0683,0.3450,121.999


In [16]:
similarity_score = create_similarity_score(redlight_df,recomms_df)

In [17]:
similarity_score.shape

(51, 250)

In [18]:
[np.argmax(i) for i in similarity_score]

[16,
 151,
 55,
 220,
 195,
 227,
 122,
 156,
 43,
 11,
 246,
 194,
 99,
 144,
 46,
 18,
 174,
 55,
 115,
 243,
 79,
 168,
 81,
 25,
 150,
 82,
 202,
 141,
 78,
 150,
 149,
 45,
 214,
 82,
 206,
 183,
 179,
 151,
 103,
 15,
 233,
 208,
 202,
 4,
 147,
 191,
 238,
 44,
 182,
 135,
 69]

In [22]:
final_recomms=recomms_df.iloc[[np.argmax(i) for i in similarity_score]]
final_recomms = final_recomms.drop_duplicates().reset_index(drop = True)

In [23]:
final_recomms

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Second Lives,Flashmob,4TM4Sehge3RjaUZvKhBFQ6,Vitalic,264706,32,0.670,0.688,1,-4.728,1,0.0430,0.001610,0.65400,0.0863,0.3470,129.960
1,Aeons - We Are All Astronauts Remix,Aeons,0parKbfhq70aUrE3aizKVL,Ryan Davis,537896,33,0.586,0.620,10,-11.000,0,0.0388,0.388000,0.85500,0.1020,0.0322,119.975
2,Twin Souls,Twin Souls,0XXWZcLfM8drFINoxPx5fC,Luttrell,269460,45,0.577,0.873,6,-6.657,1,0.0344,0.259000,0.94700,0.1180,0.2010,122.982
3,Chill Out For Strings,"WRE Sessions, Vol. 3",2RIdYrifSVDSxEKuiimcTO,Redlounge Orchestra,291371,32,0.563,0.765,10,-9.285,1,0.0307,0.005500,0.87300,0.1100,0.1660,114.001
4,Oblique,Vibrant Forms II,0O8RouQI1cT37Kta9GIiwh,Fluxion,709366,14,0.677,0.253,1,-20.243,1,0.0354,0.264000,0.82500,0.1060,0.0485,124.965
5,Kvrt in Space,Scattering,5Wdzi52wZtHB6tLfI855ON,Fraunhofer Diffraction,322795,46,0.520,0.807,6,-5.061,1,0.0534,0.003320,0.06860,0.0930,0.2560,116.037
6,Parallaxis - Traumprinz's Over 2 The End Version,Decay Versions Pt. 2,6o6LRX4HHHGKgH7NuMxw6u,Efdemin,406000,42,0.688,0.496,7,-15.994,1,0.0519,0.076400,0.89400,0.1170,0.2030,122.971
7,Epsilon Centauri,Theatre of Freaks,1vwjBiZmzNxOkskPrTvm4P,Ron Flatter,442497,38,0.695,0.613,5,-9.468,0,0.0463,0.003720,0.88200,0.1150,0.3970,122.000
8,Letting People Go (feat. Prinze George) - Dipl...,Letting People Go (feat. Prinze George) [Diplo...,3iINfVB6kucFrmMRWdLMWS,Carnage,259209,50,0.788,0.763,6,-5.032,0,0.0460,0.003370,0.04890,0.0671,0.4510,121.995
9,Poney Pt. I,OK Cowboy,6PSRMbqr5se9zwl5iOYOA8,Vitalic,323000,43,0.580,0.718,7,-6.165,1,0.0366,0.000824,0.77400,0.0746,0.5220,136.007


In [31]:
# sp_m.user_playlist_add_tracks(usernames[0],
#                               playlist_id="spotify:playlist:4a5UzxEPehS5Zoqh7lFREs",
#                               tracks = final_recomms["track_id"].tolist())

{'snapshot_id': 'NSw4OTM2NTc0ODA2OWY0OTc0NTQwYzBjYmNlNGRjODA0Nzg4YzNjNjAw'}

In [32]:
#this playlist was good but not close enough to the original

In [26]:
# mean_song["track_name"] = "avg_song"
# mean_song["album"] = "whoknows"
# mean_song["track_id"] = "notgiven"
# mean_song["artist"] = "math"
# mean_song.reset_index(drop = True, inplace = True)

In [24]:
create_similarity_score(mean_song,final_recomms) #something doesn't work here

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [25]:
def filter_with_meansong(mean_song,recommendations_df, n_recommendations = 10):
    mean_song_feat = mean_song[list(mean_song.columns[6:])].values
    mean_song_scaled = StandardScaler().fit_transform(mean_song_feat.reshape(-1,1))
    recommendations_df_scaled = StandardScaler().fit_transform(recommendations_df[features])
    mean_song_scaled = mean_song_scaled.reshape(1,-1)
    sim_mean_finrecomms = cosine_similarity(mean_song_scaled,recommendations_df_scaled)[0][:]
    #sim_mean_finrecomms = sim_mean_finrecomms[0][:]
    indices = (-sim_mean_finrecomms).argsort()[:n_recommendations]
    final_recommendations = recommendations_df.iloc[indices]
    return final_recommendations

In [26]:
features = list(mean_song.columns[6:])
mean_song_feat = mean_song[features].values

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler


mean_song_scaled = StandardScaler().fit_transform(mean_song_feat.reshape(-1,1))
mean_song_scaled = mean_song_scaled.reshape(1,-1)

In [28]:
final_recomms_scaled = StandardScaler().fit_transform(final_recomms[features])
final_recomms_scaled

array([[-0.15891754,  0.75085609, -1.34392726,  1.62749672,  0.95742711,
        -0.4892117 , -0.77372406, -0.27199015, -0.52630186,  0.30380431,
         0.83865829],
       [-0.95536032,  0.36025608,  1.28053447,  0.08607557, -1.04446594,
        -0.71265307,  0.9307027 ,  0.46255152, -0.28401782, -1.21075122,
         0.17233503],
       [-1.04069348,  1.81351787,  0.11410703,  1.15342124,  0.95742711,
        -0.94673451,  0.36166351,  0.79875965, -0.03710415, -0.39862615,
         0.37299943],
       [-1.17343394,  1.19315316,  1.28053447,  0.50755792,  0.95742711,
        -1.14357571, -0.75656467,  0.52833137, -0.16056098, -0.56701701,
        -0.22632447],
       [-0.09254731, -1.74783514, -1.34392726, -2.18550536,  0.95742711,
        -0.89353418,  0.38371929,  0.35291844, -0.2222894 , -1.13232919,
         0.50532983],
       [-1.58113679,  1.4344061 ,  0.11410703,  1.54565787,  0.95742711,
         0.0640717 , -0.76618099, -2.4112971 , -0.42290676, -0.13401193,
        -0.090

In [29]:
sim_mean_finrecomms = cosine_similarity(mean_song_scaled,final_recomms_scaled)
sim_mean_finrecomms = sim_mean_finrecomms[0][:]
indices = (-sim_mean_finrecomms).argsort()[:10]
indices

array([41, 42, 32,  4,  9, 45,  6, 17, 22, 34], dtype=int64)

In [30]:
final_recomms3 = final_recomms.iloc[indices]

In [31]:
final_recomms3

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
41,Plan of Sky,"Anthology, Pt. 2",0VYy8JlM5QjHqOMJPXC2Cb,Øfdream,267906,0,0.569,0.740,3,-10.266,1,0.0685,0.817000,0.865,0.0904,0.0387,171.898
42,Crispy Bacon,30,3PQGdUJ9GTKv0EttfvPlii,Laurent Garnier,352093,43,0.712,0.562,7,-16.716,1,0.0752,0.004910,0.796,0.1030,0.3550,130.101
32,Melange,Ultraviolet Music,44zv8LOBeI3kdwsdPIwoaR,Deepchord,581904,19,0.530,0.599,1,-17.557,1,0.0485,0.157000,0.830,0.1110,0.1200,126.011
4,Oblique,Vibrant Forms II,0O8RouQI1cT37Kta9GIiwh,Fluxion,709366,14,0.677,0.253,1,-20.243,1,0.0354,0.264000,0.825,0.1060,0.0485,124.965
9,Poney Pt. I,OK Cowboy,6PSRMbqr5se9zwl5iOYOA8,Vitalic,323000,43,0.580,0.718,7,-6.165,1,0.0366,0.000824,0.774,0.0746,0.5220,136.007
45,BREATH,DANCEHALL,2WceANzJsL37I31nEKeRva,The Blaze,229426,35,0.595,0.505,9,-10.947,1,0.0401,0.156000,0.655,0.0915,0.0380,124.985
6,Parallaxis - Traumprinz's Over 2 The End Version,Decay Versions Pt. 2,6o6LRX4HHHGKgH7NuMxw6u,Efdemin,406000,42,0.688,0.496,7,-15.994,1,0.0519,0.076400,0.894,0.1170,0.2030,122.971
17,Millenium,Eternity,6T5hiZikP7CNI2uXdWYlCg,Joachim Pastor,421935,54,0.443,0.517,6,-10.019,0,0.0542,0.037200,0.895,0.0702,0.0739,123.978
22,Ibrik,Ibrik,1U4vLO6BrDAPbBvwJSMw6n,Bonobo,350054,41,0.751,0.512,5,-12.925,0,0.0669,0.259000,0.877,0.0823,0.0862,122.997
34,Nil,Mare,5G3ZKjCHie2Ikr3I4QCQGt,Christian Löffler,410949,46,0.829,0.176,6,-23.885,0,0.0545,0.216000,0.722,0.0952,0.1430,120.008


In [32]:
final_recomms_test = filter_with_meansong(mean_song,final_recomms)

In [33]:
assert final_recomms3.equals(final_recomms_test) #function works

In [33]:
sp_m.user_playlist_add_tracks(usernames[0],
                              playlist_id="spotify:playlist:4XP9wRGPImiYYiGtsB6Dd3",
                              tracks = final_recomms3["track_id"].tolist())

{'snapshot_id': 'MTYsYWUwNDUyMmQzNjE3NTk2YzEyNDRjZjZlOTYyOGE2MjA0YjQzZDQ2NA=='}

In [ ]:
#this method seemed to work quite well